<a href="https://colab.research.google.com/github/Maks2811/data_gen_pyspark/blob/main/DE_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=1eecf229d3519ea6e9d9fb266763bd5667821a22df06c098c62b999b94ab437f
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

spark = SparkSession.builder \
        .appName("Create DataFrame from CSV") \
        .master("local[*]") \
        .getOrCreate()
csv_file_path=('/content/russian_houses.csv')
df =spark.read.csv(csv_file_path, header=True, inferSchema=True, encoding='UTF-16LE', multiLine=True)
df.show()
df.printSchema()

row_count=df.count()
print(f"Количество строк в датафрейме: {row_count}")

# Определяем функцию очистки даты постройки
def clean_year(year):
    if isinstance(year, str) and year.isdigit() and len(year) == 4:
        return year
    else:
        return None

# Регистрируем UDF
clean_year_udf = udf(clean_year, StringType())

df = df.withColumn("maintenance_year", clean_year_udf(df["maintenance_year"]))

#df_house_id = df.select('house_id')
#df_house_id.show()

df = df.fillna(0)
df = df.fillna('0')
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

df = df.withColumn("maintenance_year", col("maintenance_year").cast("int"))
df.printSchema()


year_avg = round(df.filter(df["maintenance_year"] != 0).select(avg("maintenance_year")).collect()[0][0])

print(f'Средний год постройки зданий: {year_avg}')

year_median = df.filter(df["maintenance_year"] != 0).approxQuantile("maintenance_year", [0.5], 0.01)[0]
print(f'Медианный год постройки зданий: {year_median}')

spark.stop()

+--------+------------------+------------------+----------------+--------+----------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+
|house_id|          latitude|         longitude|maintenance_year|  square|population|             region|       locality_name|             address|        full_address|communal_service_id|         description|
+--------+------------------+------------------+----------------+--------+----------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+
|       1|         44.707617|         43.006476|            1974|2 661.10|        89|Ставропольский край|село Александровское|с. Александровско...|Ставропольский кр...|                1.0|Жилой дом в , по ...|
|       2|44.706720000000004|         43.005281|            1989|3 111.10|       115|Ставропольский край|село Александровское|с. Александровско...|Ставропольски